In [91]:
import numpy as np
import pandas as pd
import json
import matplotlib
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.datasets import make_blobs
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score

•	 «Описательная статистика» - процесс расчета метрик датасета. Метрики датасета состоят из следующих параметров: общее количество значений, количество уникальных значений, количество пропущенных  значений, минимально значение, максимальное значение, среднее значение, медианное значение, стандартное отклонение, гистограмма распределения значений по каналам (10 каналов) .



In [2]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [3]:
class Description():

  def __init__(self,data):

    self.data = data

  def get_json(self):
    statictics={}
    for col in self.data.columns:
      current_col={}
      current_col ['count'] = self.data[col].count()
      current_col ['unique'] = self.data[col].nunique()
      current_col ['NaN count'] = self.data[col].shape[0] - self.data[col].count()
      if (self.data[col].dtype != 'object') and ( self.data[col].dtype != 'category' ):
        current_col ['min'] = self.data[col].min()
        current_col ['max'] = self.data[col].max()
        current_col ['mean'] = self.data[col].mean().round(3)
        current_col ['std'] = self.data[col].std().round(3)
        current_col ['median'] = self.data[col].median()
        counts, bins = np.histogram(self.data[col], bins=10)
        current_col ['hist'] = {'values of the histogram bins' : counts, 'edges of the bins' :bins}
      else:
        current_col['top'] = self.data[col].value_counts().index[0]
        current_col['freq'] = self.data[col].value_counts()[0]
      statictics[col] = current_col

    with open('descriptive_statistics.json', 'w') as f:
      json.dump(statictics, f, cls=NpEncoder)
    return statictics

In [102]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data  
y = iris.target  

df = pd.DataFrame(X, columns=iris.feature_names)  
df['target'] = y 

In [105]:
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [106]:
Description(df).get_json()

{'sepal length (cm)': {'count': 150,
  'unique': 35,
  'NaN count': 0,
  'min': 4.3,
  'max': 7.9,
  'mean': 5.843,
  'std': 0.828,
  'median': 5.8,
  'hist': {'values of the histogram bins': array([ 9, 23, 14, 27, 16, 26, 18,  6,  5,  6]),
   'edges of the bins': array([4.3 , 4.66, 5.02, 5.38, 5.74, 6.1 , 6.46, 6.82, 7.18, 7.54, 7.9 ])}},
 'sepal width (cm)': {'count': 150,
  'unique': 23,
  'NaN count': 0,
  'min': 2.0,
  'max': 4.4,
  'mean': 3.057,
  'std': 0.436,
  'median': 3.0,
  'hist': {'values of the histogram bins': array([ 4,  7, 22, 24, 37, 31, 10, 11,  2,  2]),
   'edges of the bins': array([2.  , 2.24, 2.48, 2.72, 2.96, 3.2 , 3.44, 3.68, 3.92, 4.16, 4.4 ])}},
 'petal length (cm)': {'count': 150,
  'unique': 43,
  'NaN count': 0,
  'min': 1.0,
  'max': 6.9,
  'mean': 3.758,
  'std': 1.765,
  'median': 4.35,
  'hist': {'values of the histogram bins': array([37, 13,  0,  3,  8, 26, 29, 18, 11,  5]),
   'edges of the bins': array([1.  , 1.59, 2.18, 2.77, 3.36, 3.95, 4.54, 5.

### **Metrcis** - класс для расчета основных метрик классических задач машинного обучения: регрессия, классификация, кластеризация

In [107]:
class Metric:  
    
    def __init__(self):
        self.classif = self.Classification()
        self.regr=self.Regression()
        self.clust=self.Clustering()        
    
    class Classification():


      def __init__(self, y_true, y_pred, accuracy = False, precision = False, recall = False, F1 = False, error_rate = False, FPR = False, auc = True):

        self.y_true = y_true
        self.y_pred = y_pred    
        self.accuracy = accuracy
        self.precision = precision 
        self.recall = recall
        self.F1 = F1
        self.error_rate = error_rate
        self.FPR = FPR
        self.auc = auc


      def get_metrics(self):

        metrics={}

        if self.accuracy == True :

          if len(np.unique(self.y_true)) == 2:

            acc = self.get_accuracy (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            acc = 0

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_accuracy = self.get_accuracy(temp_true,temp_pred)
              acc += temp_accuracy

            acc /= num_classes

          metrics ['accuracy'] = np.round(acc, 4)

        
        if self.precision == True :

          if len(np.unique(self.y_true)) == 2:

            prec = self.get_precision (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            prec = 0

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_precision = self.get_precision(temp_true,temp_pred)
              prec += temp_precision

            prec /= num_classes

          metrics ['precision'] = np.round(prec, 4)

        
        if self.recall == True :

          if len(np.unique(self.y_true)) == 2:

            rec = self.get_recall (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            rec = 0

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_recall = self.get_recall(temp_true,temp_pred)
              rec += temp_recall

            rec /= num_classes
          metrics ['recall'] = np.round(rec, 4)

        
        if self.F1 == True :

          if len(np.unique(self.y_true)) == 2:

            f1 = self.get_f1 (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            f1 = 0

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_f1 = self.get_f1(temp_true,temp_pred)
              f1 += temp_f1

            f1 /= num_classes

          metrics ['F1'] = np.round(f1, 4)

        if self.error_rate == True :

          if len(np.unique(self.y_true)) == 2:

            er = self.get_error_rate (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            er = 0

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_er = self.get_error_rate(temp_true,temp_pred)
              er += temp_er

            er /= num_classes

          metrics ['error rate'] = np.round(er, 4)

        if self.FPR == True :

          if len(np.unique(self.y_true)) == 2:

            fpr = self.get_fpr (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            fpr = 0

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_fpr = self.get_fpr(temp_true,temp_pred)
              fpr += temp_fpr

            fpr /= num_classes

          metrics ['False Positive Rate'] = np.round(fpr, 4)
         
        if self.auc == True :

          if len(np.unique(self.y_true)) == 2:

            auc = self.get_auc (self.y_true,self.y_pred)

          else:

            num_classes = len(np.unique(self.y_true))
            auc = {}

            for class_ in list(np.unique(self.y_true)):  

              temp_true = [1 if p == class_ else 0 for p in self.y_true]
              temp_pred = [1 if p == class_ else 0 for p in self.y_pred]
              temp_auc = self.get_auc(temp_true,temp_pred)
              auc[str(class_)] = temp_auc
              rounded_auc_dict = {k: np.round(v, 4) for k, v in auc.items()}

          metrics ['auc'] = rounded_auc_dict         

        metrics = json.dumps(metrics)
        with open('save_classification_metrics.json', 'w') as f:
          json.dump(metrics,f)

        return metrics
         

      def get_accuracy(self,true, pred):
        tp = self.true_positive(true, pred)
        tn = self.true_negative(true, pred)
        fp = self.false_positive(true, pred)
        fn = self.false_negative(true, pred)
        return (tp + tn)/(tp + tn + fp + fn)   
      
      def get_precision(self, true, pred):
        tp = self.true_positive(true, pred)
        fp = self.false_positive(true, pred)
        return tp / (tp + fp + 1e-6)

      def get_recall(self, true, pred):
        tp = self.true_positive(true, pred)
        fn = self.false_negative(true, pred)
        return tp / (tp + fn + 1e-6)

      def get_f1(self, true, pred):
        recall = self.get_recall(true, pred)
        precision = self.get_precision(true, pred)
        return (2 * recall * precision) / (recall + precision + 1e-6)

      def get_error_rate(self,true, pred):
        tp = self.true_positive(true, pred)
        tn = self.true_negative(true, pred)
        fp = self.false_positive(true, pred)
        fn = self.false_negative(true, pred)
        return (fp + fn)/(tp + tn + fp + fn) 

      def get_fpr(self, true, pred):
        tn = self.true_negative(true, pred)
        fp = self.false_positive(true, pred)
        return fp / (tn + fp + 1e-6)

      def get_auc(self, true, pred):
        return roc_auc_score(true, pred)

      
      def true_positive(self,true,pred): 
        tp = 0    
        for yt, yp in zip(true,pred):        
          if yt == 1 and yp == 1:
            tp += 1    
        return tp

      def true_negative(self, true, pred): 
        tn = 0    
        for yt, yp in zip(true, pred):        
          if yt == 0 and yp == 0:
            tn += 1    
        return tn

      def false_positive(self, true, pred): 
        fp = 0    
        for yt, yp in zip(true, pred):        
          if yt == 0 and yp == 1:
            fp += 1    
        return fp

      def false_negative(self, true, pred): 
        fn = 0    
        for yt, yp in zip(true,pred):        
          if yt == 1 and yp == 0:
            fn += 1    
        return fn    

    class Regression():


      def __init__(self, y_true, y_pred, R2 = False, MSE = False, MAE = False, MAPE = False, SMAPE = False, WAPE = False):

        self.y_true = y_true
        self.y_pred = y_pred    
        self.R2 = R2
        self.MSE = MSE 
        self.MAE = MAE
        self.MAPE = MAPE
        self.SMAPE = SMAPE
        self.WAPE = WAPE  

      def get_metrics(self):

        metrics = {}

        if self.R2 == True :  

          r2 = r2_score(self.y_true, self.y_pred) 
          metrics['R2'] = np.round(r2, 4)

        if self.MSE == True :

          mse = mean_squared_error(self.y_true, self.y_pred)
          metrics['MSE'] = np.round(mse, 4)

        if self.MAE == True:

          mae = mean_absolute_error(self.y_true, self.y_pred)
          metrics['MAE'] = np.round(mae, 4)

        if self.MAPE == True:

          mape = mean_absolute_percentage_error(self.y_true, self.y_pred)
          metrics['MAPE'] = np.round(mape, 4)

        if self.WAPE == True:

          wape = np.abs(self.y_true - self.y_pred).sum() / np.abs(self.y_true).sum()
          metrics['WAPE'] = np.round(wape, 4)

        if self.SMAPE == True:

          smape = ((np.abs(self.y_true - self.y_pred)) / (np.abs(self.y_true) + np.abs(self.y_pred)) / 2).sum() / (len(y_pred))
          metrics['SMAPE'] = np.round(smape, 4)

        metrics = json.dumps(metrics)
        with open('save_regression_metrics.json', 'w') as f:
          json.dump(metrics,f)

        return metrics

    class Clustering():


      def __init__(self, X, labels, calinski_harabasz = False, davies_bouldin = False, silhouette = False):

        self.X = X
        self.labels = labels    
        self.calinski_harabasz = calinski_harabasz
        self.davies_bouldin = davies_bouldin 
        self.silhouette = silhouette

      def get_metrics(self):
        metrics = {}

        if self.calinski_harabasz == True :  

          ch = calinski_harabasz_score(self.X, self.labels) 
          metrics['Calinski-Harabasz score'] = np.round(ch, 4)

        if self.davies_bouldin == True:

          db = davies_bouldin_score(self.X, self.labels)
          metrics['Davies-Bouldin score'] = np.round(db, 4)  

        if self.silhouette == True:

          s = silhouette_score(self.X, self.labels)
          metrics['Silhouette Coefficient'] = np.round(s, 4)   

        metrics = json.dumps(metrics)
        with open('save_clustering_metrics.json', 'w') as f:
          json.dump(metrics,f)

        return metrics             

#### тест классификации

In [80]:
test=np.array([1,1,2,1,0])
pred=np.array([0,1,2,1,2])

In [109]:
m=Metric.Classification(test,pred, accuracy=True, precision=True, recall = True, F1 = True, error_rate = True, FPR = True, auc = True)

In [120]:
m.get_metrics()

'{"accuracy": 0.7333, "precision": 0.5, "recall": 0.5556, "F1": 0.4889, "error rate": 0.2667, "False Positive Rate": 0.1667, "auc": {"0": 0.375, "1": 0.8333, "2": 0.875}}'

#### тест регрессии

In [13]:
X, y_true = make_regression(n_samples = 10, n_features=2, noise=1, random_state=42)

In [15]:
X, y_pred = make_regression(n_samples = 10, n_features=2, noise=1, random_state=42)

In [18]:
y_pred = y_pred * 1.034

In [83]:
r = Metric.Regression(y_true,y_pred, R2 = True, MSE = True, MAE = True, MAPE = True, SMAPE = True, WAPE = True)

In [116]:
r.get_metrics()

'{"R2": 0.9988, "MSE": 11.7762, "MAE": 2.837, "MAPE": 0.034, "WAPE": 0.034, "SMAPE": 0.0084}'

#### тест кластеризации

In [94]:
X, labels = make_blobs(n_samples=3000, n_features = 3, centers=3, cluster_std=1.0, random_state=42)
# y - метки кластеров, Х - признаки

In [95]:
c = Metric.Clustering(X, labels, calinski_harabasz = True, davies_bouldin = True, silhouette = True)

In [96]:
c.get_metrics()

'{"Calinski-Harabasz score": 46797.2022, "Davies-Bouldin score": 0.3603, "Silhouette Coefficient": 0.7507}'